<a href="https://colab.research.google.com/github/migub/recommender-systems/blob/main/EMB_recommender_systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# pip install deezer-python

In [31]:
## 1. Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import deezer as deezer
import time
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# New section

In [18]:
## 2. Load cleaned data from previous step
data = pd.read_csv("/content/drive/MyDrive/Recommender_Systems/cleaned_data.csv")

# Display basic info
data.info()
display(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7558834 entries, 0 to 7558833
Data columns (total 17 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   genre_id          int64 
 1   ts_listen         object
 2   media_id          int64 
 3   album_id          int64 
 4   context_type      int64 
 5   release_date      object
 6   platform_name     int64 
 7   platform_family   int64 
 8   media_duration    int64 
 9   listen_type       int64 
 10  user_gender       int64 
 11  user_id           int64 
 12  artist_id         int64 
 13  user_age          int64 
 14  is_listened       bool  
 15  listen_hour       int64 
 16  listen_dayofweek  int64 
dtypes: bool(1), int64(14), object(2)
memory usage: 929.9+ MB


,genre_id,ts_listen,media_id,album_id,context_type,release_date,platform_name,platform_family,media_duration,listen_type,user_gender,user_id,artist_id,user_age,is_listened,listen_hour,listen_dayofweek
0,25471,2016-12-01 13:00:15,222606,41774,12,2004-07-04,1,0,223,0,0,9241,55164,29,False,13,3
1,25571,2016-11-30 22:25:35,250467,43941,0,2006-03-01,2,1,171,0,0,16547,55830,30,True,22,2
2,16,2016-11-19 13:59:13,305197,48078,1,2014-07-14,2,1,149,1,1,7665,2704,29,True,13,5
3,7,2016-11-26 09:21:38,900502,71521,0,2000-10-30,0,0,240,0,1,1580,938,30,False,9,5
4,7,2016-11-05 18:02:54,542335,71718,0,2008-02-15,0,0,150,0,1,1812,2939,24,True,18,5


In [19]:
client = deezer.Client()


# Testing deezer
track_id = 222606
track = client.get_track(track_id)

# Print track details
print(f"Title: {track.title}")
print(f"Artist: {track.artist.name}")
print(f"Album: {track.album.title}")
print(f"Duration: {track.duration} seconds")

# user_id = 16547
# user = client.get_user(user_id)
#
# # Print user details
# print(f"Gender: {user.gender}")
# print(f"Country: {user.country}")


Title: Seul au monde
Artist: Malin Plaisir
Album: Malin plaisir
Duration: 223 seconds


While retrieval of track data works, user data is not as easily accessible.

[I will continue to look into this, but at the moment, I'm not finding an easy solution. It could be an authentication problem, which could be circumvented by creating an app, but currently, no new app applications are being accepted by Deezer.]

In [20]:
# This function can handle track_id and user id, but in reality, we are only getting track data.

def get_deezer_info(ids, entity_type):
    """
    Fetches information from the Deezer API for a given entity type (track or user) using the deezer package.

    Args:
        ids (list): List of entity IDs (track IDs or user IDs).
        entity_type (str): Either 'track' or 'user'.

    Returns:
        pd.DataFrame: A DataFrame containing the requested information.
    """
    client = deezer.Client()
    data_list = []

    for entity_id in ids:
        try:
            if entity_type == "track":
                track = client.get_track(entity_id)
                entity_info = {
                    "track_id": entity_id,
                    "rank": track.rank,
                    "explicit_lyrics": track.explicit_lyrics,
                    "bpm": track.bpm
                }
            elif entity_type == "user":
                user = client.get_user(entity_id)
                entity_info = {
                    "id": entity_id,
                    "gender": user.gender,
                    "country": user.country
                }
            else:
                raise ValueError("Invalid entity_type. Use 'track' or 'user'.")

            data_list.append(entity_info)

        except deezer.DeezerErrorResponse as e:
            print(f"Failed to retrieve data for {entity_type} ID {entity_id}: {e}")

    return pd.DataFrame(data_list)





In [21]:
test = data.head(20)
print(test["media_id"])


0     222606
1     250467
2     305197
3     900502
4     542335
5     542335
6     542335
7     542335
8     542335
9     542335
10    542335
11    542335
12    542341
13    542335
14    542335
15    542335
16    542335
17    542335
18    542335
19    542335
Name: media_id, dtype: int64


In [22]:
# Testing on small dataset
track_info_df = get_deezer_info(test["media_id"].unique().tolist(), "track")  # Get track data
track_info_df = track_info_df.rename(columns={"track_id": "media_id"})  # Rename column for merging
test2 = test.merge(track_info_df, on="media_id", how="left")  # Merge retrieved track data

# user_info_df = get_deezer_info((test["user_id"].unique()).tolist(), "user")
# user_info_df = user_info_df.rename(columns={"id": "user_id"})  # Rename column for merging
#test = test.merge(user_info_df, on="user_id", how="left")  # Merge retrieved track data
# print(user_info_df)
# print(test)

In [23]:
test2.head()

,genre_id,ts_listen,media_id,album_id,context_type,release_date,platform_name,platform_family,media_duration,listen_type,user_gender,user_id,artist_id,user_age,is_listened,listen_hour,listen_dayofweek,rank,explicit_lyrics,bpm
0,25471,2016-12-01 13:00:15,222606,41774,12,2004-07-04,1,0,223,0,0,9241,55164,29,False,13,3,1711,False,100.1
1,25571,2016-11-30 22:25:35,250467,43941,0,2006-03-01,2,1,171,0,0,16547,55830,30,True,22,2,70929,False,160.2
2,16,2016-11-19 13:59:13,305197,48078,1,2014-07-14,2,1,149,1,1,7665,2704,29,True,13,5,115098,False,90.1
3,7,2016-11-26 09:21:38,900502,71521,0,2000-10-30,0,0,240,0,1,1580,938,30,False,9,5,103778,False,124.9
4,7,2016-11-05 18:02:54,542335,71718,0,2008-02-15,0,0,150,0,1,1812,2939,24,True,18,5,407879,False,120.5


Now implementing the above for the whole training dataset and saving the resulting df for sharing with colleagues.

In [32]:
# With improved error handling

def get_deezer_info(ids, entity_type):
    """
    Fetches information from the Deezer API for a given entity type (track or user)
    using the deezer package. Skips to the next entity if an error is encountered.
    Limits the request rate to a maximum of 50 requests per second.

    Args:
        ids (list): List of entity IDs (track IDs or user IDs).
        entity_type (str): Either 'track' or 'user'.

    Returns:
        pd.DataFrame: A DataFrame containing the requested information.
    """
    client = deezer.Client()
    data_list = []
    last_request_time = 0  # Initialize for rate limiting

    for entity_id in ids:
        # Rate limiting
        time_since_last_request = time.time() - last_request_time
        if time_since_last_request < 0.02:
            time.sleep(0.02 - time_since_last_request)

        try:
            if entity_type == "track":
                track = client.get_track(entity_id)
                # Check for "no data" error
                if track.title is None:  # Assuming title is None when no data
                    print(f"Skipping track ID {entity_id}: No data found.")
                    continue


                entity_info = {
                    "track_id": entity_id,
                    "title": track.title,  # Add title
                    "artist_name": track.artist.name,  # Add artist name
                    "rank": track.rank,
                    "explicit_lyrics": track.explicit_lyrics,
                    "bpm": track.bpm
                }

        except Exception as e:
            print(f"Failed to retrieve data for {entity_type} ID {entity_id}: {e}")
            continue

        last_request_time = time.time()

    return pd.DataFrame(data_list)

In [ ]:
media_ids = data["media_id"].unique().tolist()


# 1. Split media_ids into chunks
chunk_size = 100
media_id_chunks = [media_ids[i:i + chunk_size] for i in range(0, len(media_ids), chunk_size)]

# 2. Process each chunk and store results, saving intermediate data
track_info_dfs = []
output_file = "/content/drive/MyDrive/Recommender_Systems/track_info_deezer.csv"  # Output CSV file

for i, chunk in enumerate(media_id_chunks):
    chunk_file = f"/content/drive/MyDrive/Recommender_Systems/track_info_chunk_{i}.csv"  # File for current chunk

    # Check if chunk file already exists
    if os.path.exists(chunk_file):
        print(f"Chunk {i} already processed, loading from file...")
        track_info_df_chunk = pd.read_csv(chunk_file)
    else:
        print(f"Processing chunk {i}...")
        track_info_df_chunk = get_deezer_info(chunk, "track")
        track_info_df_chunk.to_csv(chunk_file, index=False)  # Save chunk data

    track_info_dfs.append(track_info_df_chunk)


Processing chunk 0...
Processing chunk 1...
Processing chunk 2...
Processing chunk 3...
Processing chunk 4...
Processing chunk 5...
Processing chunk 6...
Processing chunk 7...
Processing chunk 8...
Processing chunk 9...
Processing chunk 10...
Processing chunk 11...
Processing chunk 12...
Processing chunk 13...
Processing chunk 14...
Processing chunk 15...
Failed to retrieve data for track ID 90072359: {'error': {'type': 'DataException', 'message': 'no data', 'code': 800}}
Processing chunk 16...
Processing chunk 17...
Processing chunk 18...
Processing chunk 19...
Processing chunk 20...
Processing chunk 21...
Processing chunk 22...
Processing chunk 23...
Processing chunk 24...
Processing chunk 25...
Processing chunk 26...
Processing chunk 27...
Processing chunk 28...
Processing chunk 29...
Processing chunk 30...
Processing chunk 31...
Processing chunk 32...
Processing chunk 33...
Processing chunk 34...
Processing chunk 35...
Processing chunk 36...
Processing chunk 37...
Processing chunk 

In [ ]:

# 3. Combine the resulting DataFrames and save the final output
track_info_df = pd.concat(track_info_dfs, ignore_index=True)
track_info_df.to_csv(output_file, index=False)  # Save the complete data



In [ ]:
track_info_df = track_info_df.rename(columns={"track_id": "media_id"})  # Rename column for merging

In [ ]:
# Save the DataFrame 'track_info' to Google Drive.
track_info_df.to_csv("/content/drive/MyDrive/Recommender_Systems/track_info_deezer.csv", index=False)


In [ ]:
data2 = data.merge(track_info_df, on="media_id", how="left")  # Merge retrieved track data